# MIMIC 4 data - dataset construction inputevents

Code taken from GRU-ODE-Bayes preprocessing; simplified and adapted for MIMIC 4 1.0

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [ ]:
adm = pd.read_csv("/path/processed/admissions_processed.csv")

In [ ]:
outputs = pd.read_csv('/path/icu/outputevents.csv.gz')
outputs.tail()

In [ ]:
# only choose previously selected admission ids
adm_ids=list(adm["hadm_id"])
outputs=outputs.loc[outputs["hadm_id"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(outputs["subject_id"].nunique())

In [ ]:
# get item names
item_id=pd.read_csv('/path/icu/d_items.csv.gz')
item_id_1=item_id[["itemid","label"]]
item_id_1.head()

outputs_2=pd.merge(outputs,item_id_1,on="itemid")
outputs_2.head()
print("Number of patients remaining in the database: ")
print(outputs_2["subject_id"].nunique())

In [ ]:
# take only the n most used items
n_best=15
pat_for_item=outputs_2.groupby("label")["subject_id"].nunique()
frequent_labels=pat_for_item.sort_values(ascending=False)[:n_best]
outputs_3=outputs_2.loc[outputs_2["label"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(outputs_3["subject_id"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_3.index))

print(frequent_labels)

In [ ]:
outputs_label_list=['Foley', 'Void', 'OR Urine', 'Chest Tube', 'Oral Gastric', 'Pre-Admission', 'TF Residual', 'OR EBL', 'Emesis', 'Nasogastric', 'Stool', 'Jackson Pratt', 'TF Residual Output', 'Fecal Bag', 'Straight Cath']
outputs_bis=outputs_2.loc[outputs_2["label"].isin(outputs_label_list)].copy()

print("Number of patients remaining in the database: ")
print(outputs_bis["subject_id"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_bis.index))

outputs_3=outputs_bis.copy()

In [ ]:
# Verification that all input labels have the same amounts units
outputs_3.groupby("label")["valueuom"].value_counts() 

In [ ]:
outputs_3.to_csv("/path/processed/outputs_processed.csv")